# Project 4 - CLUSTERING
UID: 404749568 , 904729596 , 704774938

In [1]:
import re
import math
import string
import operator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt; plt.rcdefaults()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import text
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from statsmodels.discrete.discrete_model import Logit
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from scipy.sparse import linalg
from sklearn.decomposition import NMF

In [2]:
remove = ('headers', 'footers', 'quotes')
final_catagories = [ 'comp.graphics', 'comp.sys.mac.hardware','comp.os.ms-windows.misc',
                    'comp.sys.ibm.pc.hardware','comp.windows.x','rec.autos','rec.motorcycles',
                    'rec.sport.baseball','rec.sport.hockey','sci.crypt','sci.electronics','sci.med',
                    'sci.space','misc.forsale','talk.politics.misc','talk.politics.guns','talk.politics.mideast',
                    'talk.religion.misc','alt.atheism','soc.religion.christian']
twenty_train = fetch_20newsgroups(subset = 'all',categories=final_catagories, shuffle=True,random_state=42)

In [3]:
new_target = np.zeros(18846)
for i in range(1,18846):
    if(twenty_train.target[i]>=5) and (twenty_train.target[i]<9):
        new_target[i] = 1
    if(twenty_train.target[i]>=9) and (twenty_train.target[i]<13):
        new_target[i] = 2
    if(twenty_train.target[i]>=13) and (twenty_train.target[i]<14):
        new_target[i] = 3
    if(twenty_train.target[i]>=14) and (twenty_train.target[i]<17):
        new_target[i] = 4
    if(twenty_train.target[i]>=17):
        new_target[i] = 5
print(new_target[1:30])
print(twenty_train.target[1:30])

[ 0.  5.  0.  0.  2.  0.  2.  2.  5.  5.  2.  5.  3.  0.  5.  2.  2.  2.
  1.  1.  1.  0.  1.  2.  4.  4.  0.  1.  0.]
[ 3 17  3  4 12  4 10 10 19 19 11 19 13  0 17 12 12 11  8  7  5  1  8 10 14
 16  1  6  0]


In [7]:
def stemming_tokenizer(text):
    stemmer = SnowballStemmer("english")
    interim = text 
    interim = "".join([a for a in interim if a not in set (string.punctuation)])
    interim = re.sub('[,.-:/()?{}*$#&]',' ',interim)
    interim = "".join(b for b in interim if ord(b) < 128)
    words = interim.split()
    stemmed = [stemmer.stem(item) for item in words]
    return stemmed
final_vect = CountVectorizer(analyzer = 'word' , tokenizer = stemming_tokenizer,stop_words = 'english',min_df = 5,max_df=0.8)
final_counts = final_vect.fit_transform(twenty_train.data)
print("Dimensions of Numerical feature vector: "+ str(final_counts.shape))
print ("Number of terms Extracted: " + str(final_counts.shape[1]))

#TFIDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(final_counts)
print("Dimensions of TF-IDF vector: "+ str(X_train_tfidf.shape))

Dimensions of Numerical feature vector: (18846, 24283)
Number of terms Extracted: 24283
Dimensions of TF-IDF vector: (18846, 24283)


In [8]:
U, s, V = linalg.svds(X_train_tfidf,k=100)
print(s)

[  3.81765997   3.82091415   3.83358793   3.83683577   3.84037065
   3.85404427   3.8670715    3.87474636   3.88791474   3.89906475
   3.90668952   3.90865025   3.91221644   3.92045755   3.9377328
   3.94804621   3.95681301   3.96117128   3.98614042   3.99278686
   3.99375524   3.99969073   4.01823888   4.02302262   4.03818263
   4.05056107   4.06517333   4.07964519   4.08211187   4.09518792
   4.12256128   4.12912362   4.13937955   4.15094987   4.16790757
   4.18014174   4.20364096   4.20832978   4.23513686   4.25070119
   4.26914119   4.28770886   4.30605735   4.32106727   4.34081218
   4.35761163   4.37765991   4.39727357   4.40993431   4.42629122
   4.43354288   4.48741765   4.49913101   4.53807685   4.55039631
   4.56995463   4.58405586   4.62445108   4.63845478   4.66931913
   4.69417199   4.73801623   4.74971926   4.78219596   4.81213052
   4.82015852   4.84838011   4.86521539   4.89190857   4.89872394
   4.94012327   4.96358448   5.00348745   5.05472061   5.0823723
   5.1422803

In [16]:
rand_array = [0]*15
for i in range(1,15):    
    svd = TruncatedSVD(n_components=i, random_state=42)
    transformed_tfidf = svd.fit_transform(X_train_tfidf)
    print("Dimensions of TF-IDF vector after LSI: "+ str(transformed_tfidf.shape))
    from sklearn.metrics import confusion_matrix
    km = KMeans(n_clusters=6, init='k-means++', max_iter=300, n_init=10)
    km.fit(transformed_tfidf)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
    print("Adjusted Rand-Index: %.3f"
          % metrics.adjusted_rand_score(new_target, km.labels_))
    print("Adjusted Mutual-Index: %.3f"
          % metrics.adjusted_mutual_info_score(new_target, km.labels_))
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(transformed_tfidf, km.labels_, sample_size=1000))
    print(confusion_matrix(new_target, km.labels_))
    rand_array[i%15] = metrics.adjusted_rand_score(new_target, km.labels_)

Dimensions of TF-IDF vector after LSI: (18846, 1)
Homogeneity: 0.013
Completeness: 0.014
V-measure: 0.014
Adjusted Rand-Index: 0.008
Adjusted Mutual-Index: 0.013
Silhouette Coefficient: 0.522
[[1120 1068  506  523  117 1369]
 [1254  617  200  780   36 1062]
 [1081  760  370  619   90 1047]
 [ 321  140   58  212   10  249]
 [ 645  634  432  324  168  691]
 [ 510  559  327  212  113  622]]
Dimensions of TF-IDF vector after LSI: (18846, 2)
Homogeneity: 0.169
Completeness: 0.172
V-measure: 0.170
Adjusted Rand-Index: 0.099
Adjusted Mutual-Index: 0.169
Silhouette Coefficient: 0.362
[[ 330 2013  449 1173  542  196]
 [ 723 1180  121  440 1475   10]
 [ 969  474  641   81 1726   76]
 [ 188   44  157    3  580   18]
 [ 402   75  983   13  906  515]
 [ 100   10 1247    0  571  415]]
Dimensions of TF-IDF vector after LSI: (18846, 3)
Homogeneity: 0.208
Completeness: 0.227
V-measure: 0.217
Adjusted Rand-Index: 0.106
Adjusted Mutual-Index: 0.208
Silhouette Coefficient: 0.303
[[   7  225 2279  459  705

In [19]:
print(rand_array)
print(np.amax(rand_array))
#best for LSI/SVD n_components = 4

[0, 0.007954893170777083, 0.09922479577625302, 0.10580570877855283, 0.11872804328122995, 0.06274374042049236, 0.10562352530662279, 0.10358053155354882, 0.10123907986327853, 0.09911833663968003, 0.10299840676238721, 0.10371403377322452, 0.07326822247043631, 0.10227982978196697, 0.10699395805803523]
0.118728043281


In [20]:
from sklearn.decomposition import NMF
rand_array = [0]*15
for i in range(1,15):
    model = NMF(n_components=i, init='random', random_state=0)
    nmf_mat= model.fit_transform(X_train_tfidf) 
    print(nmf_mat.shape)
    km = KMeans(n_clusters=6, init='k-means++', max_iter=300, n_init=10)
    km.fit(nmf_mat)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
    print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(new_target, km.labels_))
    print("Adjusted Mutual-Index: %.3f"
      % metrics.adjusted_mutual_info_score(new_target, km.labels_))
    print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(transformed_tfidf, km.labels_, sample_size=1000))
    print(confusion_matrix(new_target, km.labels_))
    rand_array[i%15] = metrics.adjusted_rand_score(new_target, km.labels_)

(18846, 1)
Homogeneity: 0.014
Completeness: 0.014
V-measure: 0.014
Adjusted Rand-Index: 0.008
Adjusted Mutual-Index: 0.013
Silhouette Coefficient: -0.095
[[1090 1105  108 1411  502  487]
 [ 632 1265   34 1098  193  727]
 [ 773 1083   86 1074  367  584]
 [ 143  320   10  263   55  199]
 [ 646  641  163  716  429  299]
 [ 567  507  110  636  324  199]]
(18846, 2)
Homogeneity: 0.166
Completeness: 0.170
V-measure: 0.168
Adjusted Rand-Index: 0.088
Adjusted Mutual-Index: 0.166
Silhouette Coefficient: -0.020
[[ 367 1654  237  471 1255  719]
 [1376  705   25  313 1259  271]
 [1331  231  188 1245  942   30]
 [ 529    5   35  283  135    3]
 [ 886   26  619 1141  217    5]
 [ 394    6  542 1376   25    0]]
(18846, 3)
Homogeneity: 0.239
Completeness: 0.262
V-measure: 0.250
Adjusted Rand-Index: 0.119
Adjusted Mutual-Index: 0.239
Silhouette Coefficient: 0.034
[[ 617 2088    3   15  592 1388]
 [1977 1334   10   65   54  509]
 [1127  841  613 1062  230   94]
 [ 774   92    0    2  118    4]
 [1351  1

In [22]:
print(rand_array)
print(np.amax(rand_array))
#best for NMF n_components = 11

[0, 0.00799994272591918, 0.0880332896157403, 0.11944725147401135, 0.10693214175872967, 0.07847128202412275, 0.05137767867217855, 0.0575279814969857, 0.08837115991450478, 0.10330021879126554, 0.09522083012595255, 0.12622186302488492, 0.04532756078395182, 0.0615403348817604, 0.05514706285822128]
0.126221863025
0.12622186302488492


In [23]:
tfidf_transformer_log = TfidfTransformer(sublinear_tf = True)
X_train_tfidf_log = tfidf_transformer_log.fit_transform(final_counts)

In [25]:
model = NMF(n_components=11, init='random', random_state=0)
nmf_mat_log= model.fit_transform(X_train_tfidf_log)
km = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10)
km.fit(nmf_mat_log)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(new_target, km.labels_))
print("Adjusted Mutual-Index: %.3f"
      % metrics.adjusted_mutual_info_score(new_target, km.labels_))
#print(confusion_matrix(new_target, km.labels_))

Homogeneity: 0.444
Completeness: 0.268
V-measure: 0.334
Adjusted Rand-Index: 0.140
Adjusted Mutual-Index: 0.267
